In [ ]:
# Se importan las librerias correspondientes
import nltk
from nltk.corpus import stopwords
import pandas
import numpy as np
import utils_pln as utl # Archivo con utilitarios, los mismos se separaron del notebook para no sobrecargarlo
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation

### 1. Importamos el corpus de comentarios

Pandas es una biblioteca de código abierto implementada en Python la cual permite realizar una fácil manipulación y análisis de los datos.
Ésta se utilizó para cargar los datos en memoria y realizar un breve análisis de los mismos.

In [ ]:
# Se carga en 'datos' el archivo csv en memoria
datos = pandas.read_csv("comentarios_peliculas.csv", skiprows=1, delimiter=';', skip_blank_lines=True, encoding='utf-8')

### 2. Características del corpus
Para conocer acerca del corpus sobre el cual se trabajará se realiza un breve análisis de los datos. Para ello se obtiene la cantidad de filas y columnas que el dataset posee; donde las filas corresponden a los comentarios y las columnas a los atributos asociados a los mismos. Por otro lado se obtiene la cantidad de peliculas del corpus, la cantidad de comentarios para cada película, en el que se realiza un ploteo, entre otros. A su vez se listan esos atributos junto a su tipo asociado. Una vez hecho esto se procede a chequear de que los atributos de los comentarios tengan o no valores faltantes, ya que en caso de tenerlos hay que tomar una decisión en base a que hacer con los mismos (existen varias técnicas para este tratamiento). 

In [ ]:
# Se obtiene cantidad peliculas, de comentarios y cantidad de atributos
cantPeliculas = len(datos.ix[:,0].unique())
cantComentarios = len(datos.index)
cantAtributos = len(datos.columns)

# Se imprimen los datos obtenidos anteriormente junto a sus tipos de datos
print ("Hay una cantidad de " + str(cantPeliculas) + " películas en el corpus.")
print ("El corpus posee " + str(cantComentarios) + " comentarios con " + str(cantAtributos) + " atributos por cada uno de ellos.")

# Se imprime la cantidad de comentarios para cada pelicula
print ("\nCantidad de comentarios asociados a cada película:\n")
print (datos.ix[:,0].value_counts())

In [ ]:
# Se importa la libreria para utilizar plot
# Además se plotea en el própio notebook y no como una ventana nueva
%matplotlib inline
from pylab import *

# Se realiza una gráfica de cantidad de comentarios en función de su clasificación
gruposCalificacion = datos.groupby([u'Calificación']).groups
keyClasificacion = gruposCalificacion.keys()
cantClasificacion = []
for k in keyClasificacion:
    cantClasificacion.append(len(gruposCalificacion.get(k)))    
pos = arange(len(keyClasificacion)) + 0.5 

figure(1)
barh(pos,cantClasificacion, align='center')
yticks(pos, list(keyClasificacion))
xlabel('Cantidad de comentarios')
ylabel(u'Clasificación')
title(u'Grafica cantidad de comentarios por clasificación')
grid(True)
show()

In [ ]:
#Se imprime la cantidad de comentarios asociados a cada clase (es decir cantidad de comentarios por calificación)
print ("\nCantidad de comentarios asociados a cada calificación:\n")
print (datos.ix[:,7].value_counts())

In [ ]:
print ("A continuación se listan los atributos con su tipo asociado:\n")
print (datos.dtypes)

In [ ]:
# Se chequea si los atributos poseen o no valores faltantes
# Para ello se recorren todos los atributos (columnas de 'datos')
for atributo in datos.columns:
    
    # Se obtiene la cantidad de valores distinto de vacio del atributo 
    cantValoresAtributo = datos[atributo].describe()['count']
    
    # Si hay menos que la cantidad de comentarios de 'datos' -> hay valores faltantes
    if(cantComentarios > cantValoresAtributo):
        print ("Hay valores faltantes para el atributo " + atributo)
    else:
        print ("No hay valores faltantes para el tributo " + atributo)

### 3. Descartamos columnas innecesarias
En esta sección se descartan las columnas (atributos) que se consideran innecesarios. En particular las columnas que creemos se consideran necesarias son solamente las columnas de los comentarios y los puntajes asociados a dichos comentarios. Por lo que se procede a eliminar el resto.

In [ ]:
# Se descartan de 'datos' las columnas innecesarias, dejando solamente los comentarios y sus calificaciones
datos.drop(datos.columns[[0,1,2,3,4,5,8]],inplace=True,axis=1)
comentarios_peliculas = datos
# Se imprime un resumen de los datos
print (comentarios_peliculas)

### 3. Preprocesamiento de los comentarios
El corpus sobre el cual se trabaja puede contener datos con ruido en su contenido. Es decir, diversos elementos que están mezclados en el contenido pero que además de no aportan nada a la tarea de clasificación, generan ruido dificultando las tareas posteriores. Para ello entonces se procede a realizar el preprocesamiento de los comentarios.

In [ ]:
# Se limpian los datos y se los convierte a una lista de tuplas para su facil manipualacion.
comentarios_peliculas = utl.convert_to_list(utl.depurar_comentarios(comentarios_peliculas))

### 4. Separamos entrenamiento y testeo 
Una vez de haber llegado a este punto se divide dicha lista en conjuntos de entrenamiento y testeo, para primero entrenar el algoritmo y luego testearlo.
Para poder entrenar y testear un algoritmo de aprendizaje, como se mencionó, es necesario primero particionar los datos en dos conjuntos disjuntos de entrenamiento y testeo. Se separarán aleatoriamente un 20% de los datos para testeo y el 80% restante se utilizará para el entrenamiento. Para esto se utilizó la función train_test_split del paquete cross_validation de sklearn.

In [ ]:
# Se divide el conjunto de datos en train (80%) y test (20%)
datos_train, datos_test = train_test_split(comentarios_peliculas, test_size=0.2)

# Se chequean las cantidades de la división
print (len(datos_train))
print (len(datos_test))

### 4. Tokenización
En esta sección se realiza el proceso de tokenización de los textos utilizando la biblioteca NLTK. Además se convierte a un formato suceptible para la clasificación. Es decir, se toquenizan los comentarios y se calcula el diccionario de palabras frecuentes del mismo. En base a ello y al valor de clasificación asociado se crea una tupla la cual corresponde al comentario analizado. Luego estas tuplas (una por cada comentario) son la entrada para el entrenamiento del algoritmo Entropía Máxima provisto por NLTK.

In [ ]:
datos_train_tokenizados = utl.tokenizar_nltk(datos_train)

#Se imprime el primer comentario tokenizado, cada palabra aparece junto a la cantidad de veces
# que ocurre en el comentario y ademas se imprime la clasificacion del comentario.
print (datos_train_tokenizados[0])

### 6. Se entrena el algoritmo de máxima entropia y se lo evalua
Una vez de tener los datos en un formato aceptable para los algoritmos de entrenamiento y particionado los mismos en test y train, se procede a realizar el entrenamiento del algoritmo de Máxima Entropía y se utiliza cross validation. Con cada clasificador de la cross validation se calcula el promedio de la acuracy. En este caso, es decir utilizando como features todas las palabras se obtiene un 84%.

Obs: Este proceso es muy lento (aproximadamente 25 minutos)

In [ ]:
cv = cross_validation.KFold(len(datos_train_tokenizados), n_folds=10)
    
accuracy = 0
for traincv, evalcv in cv:
    clf_max_ent = nltk.classify.MaxentClassifier.train(datos_train_tokenizados[traincv[0]:traincv[len(traincv)-1]],max_iter=8)
    accuracy += nltk.classify.util.accuracy(clf_max_ent, datos_train_tokenizados[evalcv[0]:evalcv[len(evalcv)-1]])

accuracy_avg = (accuracy / 10)
print('ACCURACY: ' + str(accuracy_avg))

#### Evaluacion del clasificador con los datos de test
Luego del entrenamiento se procede a evaluar el clasificador con los datos de test, y se obtiene la matriz de confusion para poder analizar claramente los errores (Esto se realiza con la funcion getTasa)

In [ ]:
# En la siguiente linea se entrena el algoritmo con 8 iteraciones
# Obs. El entrenamiento es bastante lento.
clf_max_ent = nltk.classify.MaxentClassifier.train(datos_train_tokenizados,max_iter=8)

In [ ]:
#Se imprime la tasa de acierto y la matriz de confucion
datos_test_tokenizado = utl.tokenizar_nltk(datos_test)
utl.getTasa(clf_max_ent,datos_test_tokenizado)

Como se puede ver en la matriz de confucion los resultados aun son poco fiables, por ejemplo en el caso de los comentarios negativos la mayoria de los comentarios fueron mal clasificados.

####Palabras mas frecuentes
En esta seccion se busca mejorar el clasificaodr utilizando como features las palabras mas frecuentes, para esto se cuentan las palabras del corpus y se calcula su frecuencia. Se probara el clasificador para distintas cantidadades de palabras.

In [ ]:
# Cross valodation para el clasificador
# Frecuencia -1 significa todas las palabras
fecuencias = [-1, 5000, 3000, 2000, 1000, 500]

accuracy_frec_cv = []

for frec in fecuencias:
    print('******************************************************************************')
    print("FRECUENCIA: " + str(frec))
    
    datos_train_frec = utl.filtrar(datos_train, utl.palabras_mas_frecuentes(frec,datos_train), False)
    datos_train_tokenizados = utl.tokenizar_nltk(datos_train_frec)
    
    cv = cross_validation.KFold(len(datos_train_tokenizados), n_folds=10)
    
    accuracy = 0
    for traincv, evalcv in cv:
        classifier = nltk.classify.MaxentClassifier.train(datos_train_tokenizados[traincv[0]:traincv[len(traincv)-1]],max_iter=8,trace=1)
        accuracy += nltk.classify.util.accuracy(classifier, datos_train_tokenizados[evalcv[0]:evalcv[len(evalcv)-1]])
        
    accuracy_avg = (accuracy / 10)
    accuracy_frec_cv += [(frec,accuracy_avg)]
    print('ACCURACY: ' + str(accuracy_avg))
print (accuracy_frec_cv)

In [ ]:
# Corroborar los resultados de CV con todos el set de test
# Cross valodation para el clasificador
# Frecuencia -1 significa todas las palabras
fecuencias = [-1, 5000, 3000, 2000, 1000, 500]
accuracy_frec = []
for frec in fecuencias:
    print('******************************************************************************')
    print("FRECUENCIA: " + str(frec))
    
    datos_train_frec = utl.filtrar(datos_train, utl.palabras_mas_frecuentes(frec,datos_train), False)
    datos_train_tokenizados = utl.tokenizar_nltk(datos_train_frec)
    
    classifier = nltk.classify.MaxentClassifier.train(datos_train_tokenizados,max_iter=8,trace=1)
    accuracy_frec += [(frec,utl.getTasa(classifier,datos_test_tokenizado))]
print (accuracy_frec)

In [ ]:
# Frecuencia -1 significa todas las palabras
fecuencias = [-1, 5000, 3000, 2000, 1000, 500]

accuracy_frec_cv = []

dominio_cine_peliculas = open('terminosNoValorativosAmbitoCine.txt').read()
caracteres_especiales = open('caracteres_especiales.txt').read()
nltk_stopwords = stopwords.words('spanish')

datos_train_frec = utl.filtrar(datos_train, nltk_stopwords, True)
datos_train_frec = utl.filtrar(datos_train_frec, caracteres_especiales, True)
datos_train_frec = utl.filtrar(datos_train_frec, dominio_cine_peliculas, True)

for frec in fecuencias:
    print('******************************************************************************')
    print("FRECUENCIA: " + str(frec))
    
    datos_train_frec = utl.filtrar(datos_train_frec, utl.palabras_mas_frecuentes(frec,datos_train), False)
    datos_train_tokenizados = utl.tokenizar_nltk(datos_train_frec)
    
    cv = cross_validation.KFold(len(datos_train_tokenizados), n_folds=10)
    
    accuracy = 0
    for traincv, evalcv in cv:
        classifier = nltk.classify.MaxentClassifier.train(datos_train_tokenizados[traincv[0]:traincv[len(traincv)-1]],max_iter=8,trace=1)
        accuracy += nltk.classify.util.accuracy(classifier, datos_train_tokenizados[evalcv[0]:evalcv[len(evalcv)-1]])
        
    accuracy_avg = (accuracy / 10)
    accuracy_frec_cv += [(frec,accuracy_avg)]
    print('ACCURACY: ' + str(accuracy_avg))
print (accuracy_frec_cv)

In [ ]:
# Frecuencia -1 significa todas las palabras
fecuencias = [-1, 5000, 3000, 2000, 1000, 500]

accuracy_frec_cv = []

dominio_cine_peliculas = open('terminosNoValorativosAmbitoCine.txt').read()
caracteres_especiales = open('caracteres_especiales.txt').read()
nltk_stopwords = stopwords.words('spanish')

datos_train_frec = utl.filtrar(datos_train, nltk_stopwords, True)
datos_train_frec = utl.filtrar(datos_train_frec, caracteres_especiales, True)
datos_train_frec = utl.filtrar(datos_train_frec, dominio_cine_peliculas, True)

for frec in fecuencias:
    print('******************************************************************************')
    print("FRECUENCIA: " + str(frec))
    
    datos_train_frec = utl.filtrar(datos_train_frec, utl.palabras_mas_frecuentes(frec,datos_train), False)
    datos_train_tokenizados = utl.tokenizar_nltk(datos_train_frec)
    
    classifier = nltk.classify.MaxentClassifier.train(datos_train_tokenizados,max_iter=8,trace=1)
    accuracy_frec_cv += [(frec,utl.getTasa(classifier,datos_test_tokenizado))]
    
    accuracy_frec_cv += [(frec,accuracy_avg)]
    print('ACCURACY: ' + str(accuracy_avg))
print (accuracy_frec_cv)

In [ ]:
def codificarClasificacionesSubjetivos():
    lista = []
    for t in elementos_subjetivos:

        clasificacion = 'neg'
        if elementos_subjetivos[t] == 3:
            clasificacion = 'pos'
            
        lista.append(({t:1},clasificacion))
    
    return lista

def getPositivos():
    lista = []
    for t in elementos_subjetivos:

        #clasificacion = 'neg'
        if elementos_subjetivos[t] == 3:
            lista.append(t)
    
    return lista

def getNegativos():
    lista = []
    for t in elementos_subjetivos:

        #clasificacion = 'neg'
        if elementos_subjetivos[t] != 3:
            lista.append(t)
    
    return lista

In [ ]:
#print(codificarClasificacionesSubjetivos())
#clf_max_ent.train(codificarClasificacionesSubjetivos(), max_iter = 7)

In [ ]:
subPositivos = getPositivos()
subNegativos = getNegativos()


In [ ]:
def atributos(diccionario,SUBP,SUBN):
    atributos = {}
    cantPositivos = 0
    cantNegativos = 0
    cantPalabrasNeutras = 0
    comentario = list(diccionario.keys())[0]
    largoComentario = len(comentario)
    #GRAL
    
    if (largoComentario > 14): 
        atributos["comentario_largo"]=True
        atributos["comentario_corto"]=False
    else: 
        atributos["comentario_corto"]=True
        atributos["comentario_largo"]=False

    for w in SUBP:
        if w in comentario: atributos[w]=True
        else: atributos[w]=False
        cantPositivos += comentario.count(w)
    for w in SUBN:
        if w in comentario: atributos[w]=True
        else: atributos[w]=False
        cantNegativos += comentario.count(w)

    #POSITIVOS
    if (cantPositivos < 1): atributos["hay_Positivos"] = False
    else: atributos["hay_Positivos"] = True

    if (cantPositivos > 3): atributos["muchos_Positivos"] = True
    else: atributos["muchos_Positivos"] = False

    #NEGATIVOS
    if (cantNegativos < 1): atributos["hay_Negativos"] = False
    else: atributos["hay_Negativos"] = True

    if (cantNegativos > 3): atributos["muchos_Negativos"] = True
    else: atributos["muchos_Negativos"] = False

    #NEUTRAS
    for w in comentario:
        if ((w not in SUBN) & (w not in SUBP)): 
            #atributos[w] = 1
            cantPalabrasNeutras +=1

    if (cantPalabrasNeutras <= 5): atributos["pocasPalabrasNeutras"] = True
    else: atributos["pocasPalabrasNeutras"] = False

    return atributos


In [ ]:
training_set = [(atributos(c, subPositivos, subNegativos), p) for (c,p) in datos_train]

In [ ]:
test_set = [(atributos(c, subPositivos, subNegativos), p) for (c,p) in datos_test]

In [ ]:
clf_max_ent = nltk.classify.MaxentClassifier.train(training_set,max_iter=7)
print(nltk.classify.accuracy(clf_max_ent, test_set))
clf_max_ent.show_most_informative_features()

In [ ]:
salidaClasificador = []
salida = []
for c in test_set:
    salidaClasificador.append(clf_max_ent.classify(c[0]))
    salida.append(c[1])

cm = nltk.ConfusionMatrix(salidaClasificador, salida)
print(cm)
